# Hard voting

Tests of the hard voting for chosen models.

In [30]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd

In [22]:
seed = 2024
TRAIN_DIR = '../voting/data/train'

In [23]:
def hard_vote(preds1, preds2, preds3):
    final_preds = []
    for i in range(len(preds1)):
        combined_preds = np.array([preds1[i], preds2[i], preds3[i]])
        final_preds.append(np.argmax(np.sum(combined_preds, axis=0)))
    return np.array(final_preds)

## Test 1: Hard voting for best VGG models
VGG8 for learning rate = 0.01 $\newline$
VGG9 for batch size = 128 $\newline$
VGG14

In [24]:
vgg8 = load_model('vgg8/3.keras')
vgg9 = load_model('vgg9/2.keras')
vgg14 = load_model('vgg14/0.keras')

In [25]:
datagen = ImageDataGenerator(validation_split=0.2,
                             rescale=1./255)

val_generator = datagen.flow_from_directory(
        TRAIN_DIR,
        subset='validation',
        seed=seed,
        target_size=(32, 32),
        batch_size=16
    )

Found 18000 images belonging to 10 classes.


In [26]:
X, y = zip(*(val_generator[i] for i in range(len(val_generator))))
X_test, y_test = np.vstack(X), np.vstack(y)

In [27]:
y_pred1 = vgg8.predict(X_test)
y_pred2 = vgg9.predict(X_test)
y_pred3 = vgg14.predict(X_test)
y_test = np.argmax(y_test, axis=1)

563/563 [==============================] - 160s 282ms/step


In [28]:
final_predictions = hard_vote(y_pred1, y_pred2, y_pred3)
accuracy = accuracy_score(final_predictions, y_test)
accuracy

0.5692222222222222

In [36]:
name = 'vgghardvoting'
accuracy_df = pd.DataFrame(data=[accuracy], columns = ['accuracy'])
accuracy_csv_file = f'train_history/{name}/accuracy.csv'
with open(accuracy_csv_file, mode='w') as f:
    accuracy_df.to_csv(f)
print(f'Attempts accuracy is saved to {accuracy_csv_file}')

Attempts accuracy is saved to train_history/vgghardvoting/accuracy.csv
